# Upvoting if it helps🔥🔥🔥

## pytorch part

In [1]:
import os
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import log_loss, roc_auc_score
import datatable as dt

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

DATA_PATH = '../input/jane-street-market-prediction/'

NFOLDS = 5

TRAIN = False
CACHE_PATH = './model'

def save_pickle(dic, save_path):
    with open(save_path, 'wb') as f:
    # with gzip.open(save_path, 'wb') as f:
        pickle.dump(dic, f)

def load_pickle(load_path):
    with open(load_path, 'rb') as f:
    # with gzip.open(load_path, 'rb') as f:
        message_dict = pickle.load(f)
    return message_dict

feat_cols = [f'feature_{i}' for i in range(130)]

target_cols = ['action', 'action_1', 'action_2', 'action_3', 'action_4']

f_mean = np.load(f'{CACHE_PATH}/f_mean_online.npy')

##### Making features
all_feat_cols = [col for col in feat_cols]
all_feat_cols.extend(['cross_41_42_43', 'cross_1_2'])

##### Model&Data fnc
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(all_feat_cols))
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(len(all_feat_cols), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(all_feat_cols), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

if True:
    device = torch.device("cuda:0")

    model_list = []
    tmp = np.zeros(len(feat_cols))
    for _fold in range(NFOLDS):
        torch.cuda.empty_cache()
        model = Model()
        model.to(device)
        model_weights = f"{CACHE_PATH}/online_model{_fold}.pth"
        model.load_state_dict(torch.load(model_weights))
        model.eval()
        model_list.append(model)

## tensorflow part

In [2]:
!ls ../input/jane-street-with-keras-nn-overfit

ls: 无法访问../input/jane-street-with-keras-nn-overfit: 没有那个文件或目录


In [3]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices


SEED = 1111

np.random.seed(SEED)

# fit
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

epochs = 200
batch_size = 4096
hidden_units = [160, 160, 160]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

tf.keras.backend.clear_session()
tf.random.set_seed(SEED)
clf = create_mlp(
    len(feat_cols), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )
clf.load_weights('./model/model.h5')

tf_models = [clf]

# Pytorch Prediction

In [4]:
# train = pd.read_csv(f'{DATA_PATH}/train.csv')
# train = dt.fread(f'{DATA_PATH}/train.csv').to_pandas()
train = dt.fread('/kaggle/working/input/train.csv').to_pandas()
valid = train.loc[(train.date >= 450) & (train.date < 500)].reset_index(drop=True)
valid.fillna(train.mean(), inplace=True)
del train

In [5]:
valid['action'] = (valid['resp'] > 0).astype('int')
valid['action_1'] = (valid['resp_1'] > 0).astype('int')
valid['action_2'] = (valid['resp_2'] > 0).astype('int')
valid['action_3'] = (valid['resp_3'] > 0).astype('int')
valid['action_4'] = (valid['resp_4'] > 0).astype('int')

In [6]:
valid['cross_41_42_43'] = valid['feature_41'] + valid['feature_42'] + valid['feature_43']
valid['cross_1_2'] = valid['feature_1'] / (valid['feature_2'] + 1e-5)

In [7]:
BATCH_SIZE = 8192
class MarketDataset:
    def __init__(self, df):
        self.features = df[all_feat_cols].values

        self.label = df[target_cols].values.reshape(-1, len(target_cols))

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return {
            'features': torch.tensor(self.features[idx], dtype=torch.float),
            'label': torch.tensor(self.label[idx], dtype=torch.float)
        }

valid_set = MarketDataset(valid)
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [8]:
def inference_fn(model, dataloader, device):
    model.eval()
    preds = []

    for data in dataloader:
        features = data['features'].to(device)

        with torch.no_grad():
            outputs = model(features)

        preds.append(outputs.sigmoid().detach().cpu().numpy())

    preds = np.concatenate(preds).reshape(-1, len(target_cols))

    return preds

In [9]:
pt_valid_pred = np.zeros((len(valid), len(target_cols)))
for model in model_list:
    pt_valid_pred += inference_fn(model, valid_loader, device) / len(model_list)

In [10]:
pt_valid_pred = np.median(pt_valid_pred, axis=1)
pt_valid_pred

(278315,)

# Tensorflow Prediction

In [11]:
X_valid = valid.loc[:, feat_cols]
# y_valid = np.stack([(valid[c] > 0).astype('int') for c in target_cols]).T
import gc
gc.collect()

44

In [12]:
tf_valid_pred = np.zeros((len(valid), len(target_cols)))
for model in tf_models:
    tf_valid_pred += model(X_valid.values).numpy() / len(tf_models)

In [13]:
tf_valid_pred = np.median(tf_valid_pred, axis=1)
tf_valid_pred

array([0.49571002, 0.49935651, 0.48506093, ..., 0.50859487, 0.48930159,
       0.32039496])

# Searching

In [18]:
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [27]:
best_threshold, best_u_score = 0.5, 0
for i in range(4500, 5500):
    thres = float(i) / 10000
    pt_slice_valid_pred = pt_valid_pred.copy()
    tf_slice_valid_pred = tf_valid_pred.copy()
    slice_valid_pred = 0.5 * pt_slice_valid_pred + 0.5 * tf_slice_valid_pred
    slice_valid_pred = np.where(slice_valid_pred >= thres, 1, 0).astype(int)
    valid_u_score = utility_score_bincount(date=valid.date.values, weight=valid.weight.values,
                                           resp=valid.resp.values, action=slice_valid_pred)
    print(f'thresold={thres:.4f}, valid_u_score={valid_u_score:.4f}')
    
    if valid_u_score >= best_u_score:
        best_u_score = valid_u_score
        best_threshold = thres

thresold=0.4500, valid_u_score=4077.6200
thresold=0.4501, valid_u_score=4085.6298
thresold=0.4502, valid_u_score=4095.2351
thresold=0.4503, valid_u_score=4098.4939
thresold=0.4504, valid_u_score=4114.0605
thresold=0.4505, valid_u_score=4118.8160
thresold=0.4506, valid_u_score=4112.0570
thresold=0.4507, valid_u_score=4116.9741
thresold=0.4508, valid_u_score=4113.7114
thresold=0.4509, valid_u_score=4107.4407
thresold=0.4510, valid_u_score=4118.6784
thresold=0.4511, valid_u_score=4129.1324
thresold=0.4512, valid_u_score=4133.3140
thresold=0.4513, valid_u_score=4132.6546
thresold=0.4514, valid_u_score=4131.0756
thresold=0.4515, valid_u_score=4150.4821
thresold=0.4516, valid_u_score=4161.5374
thresold=0.4517, valid_u_score=4165.8879
thresold=0.4518, valid_u_score=4160.7731
thresold=0.4519, valid_u_score=4165.2402
thresold=0.4520, valid_u_score=4181.6249
thresold=0.4521, valid_u_score=4193.5973
thresold=0.4522, valid_u_score=4198.5768
thresold=0.4523, valid_u_score=4199.1790
thresold=0.4524,

In [28]:
print(f'Best thresold={best_threshold:.4f}, best valid u score={best_u_score:.4f}')

Best thresold=0.4973, best valid u score=5958.8781


## Inference

In [ ]:
if True:
    import janestreet
    env = janestreet.make_env()
    env_iter = env.iter_test()

    for (test_df, pred_df) in tqdm(env_iter):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, feat_cols].values
            if np.isnan(x_tt.sum()):
                x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean

            cross_41_42_43 = x_tt[:, 41] + x_tt[:, 42] + x_tt[:, 43]
            cross_1_2 = x_tt[:, 1] / (x_tt[:, 2] + 1e-5)
            feature_inp = np.concatenate((
                x_tt,
                np.array(cross_41_42_43).reshape(x_tt.shape[0], 1),
                np.array(cross_1_2).reshape(x_tt.shape[0], 1),
            ), axis=1)

            # torch_pred
            torch_pred = np.zeros((1, len(target_cols)))
            for model in model_list:
                torch_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / NFOLDS
            torch_pred = np.median(torch_pred)
            
            # tf_pred
            tf_pred = np.median(np.mean([model(x_tt, training = False).numpy() for model in tf_models],axis=0))
            
            # avg
            pred = torch_pred * 0.5 + tf_pred * 0.5
            
            pred_df.action = np.where(pred >= 0.4916, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)